In [1]:
#%pip install pyodbc

In [1]:
print("Hello, World!")

Hello, World!


In [2]:
import pyodbc
import sys

# SQL Server connection configuration (from your screenshot)
SERVER = 'DESKTOP-6IHV0U9\\SAURABH'  # Note the double backslash
DATABASE = 'ExpenseTrackerDB'
USERNAME = 'sa'
PASSWORD = '1234'  # Replace with your actual password

# SQL Server Authentication
CONNECTION_STRING = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={SERVER};DATABASE={DATABASE};Trusted_Connection=yes;'
drivers = pyodbc.drivers()
conn = pyodbc.connect(CONNECTION_STRING, timeout=5)
print("✓ Connection successful!\n")

✓ Connection successful!



In [3]:
cursor = conn.cursor()

# Create table
cursor.execute("""
    CREATE TABLE test_table (
        id INT PRIMARY KEY,
        name NVARCHAR(100)
    )
""")
conn.commit()

# Insert record
cursor.execute("INSERT INTO test_table (id, name) VALUES (1, 'Test Record')")
conn.commit()

# Verify
cursor.execute("SELECT * FROM test_table")
for row in cursor.fetchall():
    print(f"ID: {row.id}, Name: {row.name}")

# Clean up
cursor.execute("DROP TABLE test_table")


ID: 1, Name: Test Record


In [4]:
cursor.execute("DROP TABLE test_table")


ProgrammingError: ('42S02', "[42S02] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Cannot drop the table 'test_table', because it does not exist or you do not have permission. (3701) (SQLExecDirectW)")

In [5]:
cursor.close()
conn.close()

In [ ]:
cursor.execute("SELECT SYSTEM_USER, USER_NAME()")
user_info = cursor.fetchone()

# List databases
cursor.execute("SELECT name FROM sys.databases ORDER BY name")

Error: ('HY000', '[HY000] [Microsoft][ODBC Driver 17 for SQL Server]Connection is busy with results for another command (0) (SQLExecDirectW)')

In [10]:
conn.cursor().execute("SELECT @@VERSION;")

In [ ]:

        # Get SQL Server version
        cursor = conn.cursor()
        cursor.execute("SELECT @@VERSION AS Version")
        version = cursor.fetchone()[0]
        print("SQL Server Version:")
        print("-" * 60)
        print(version)
        print()
        
        # Get database name
        cursor.execute("SELECT DB_NAME() AS DatabaseName")
        db_name = cursor.fetchone()[0]
        print(f"Connected to database: {db_name}\n")

        # List tables in the database
        cursor.execute("""
            SELECT TABLE_NAME 
            FROM INFORMATION_SCHEMA.TABLES 
            WHERE TABLE_TYPE = 'BASE TABLE'
            ORDER BY TABLE_NAME
        """)
        tables = cursor.fetchall()
        
        print("Tables in database:")
        print("-" * 60)
        if tables:
            for table in tables:
                print(f"  - {table[0]}")
        else:
            print("  (No tables found)")
        print()
        
        # Close connection
        cursor.close()
        conn.close()
        
        print("=" * 60)
        print("✓ TEST PASSED - Connection is working properly!")
        print("=" * 60)
        return True
        
    except pyodbc.Error as e:
        print("✗ CONNECTION FAILED\n")
        print("Error Details:")
        print("-" * 60)
        print(f"Error: {str(e)}\n")
        
        print("Troubleshooting Tips:")
        print("-" * 60)
        print("1. Verify the password for 'sa' user is correct\n")
        
        print("2. Check if SQL Server Authentication is enabled:")
        print("   - Open SQL Server Management Studio")
        print("   - Right-click server → Properties → Security")
        print("   - Ensure 'SQL Server and Windows Authentication mode' is selected\n")
        
        print("3. Verify SQL Server is running:")
        print("   - Open SQL Server Configuration Manager")
        print("   - Check if SQL Server (SAURABH) service is running\n")
        
        print("4. Create the database if it doesn't exist:")
        print("   - Open SSMS and connect")
        print("   - Run: CREATE DATABASE ExpenseTrackerDB;\n")
        
        print("5. Enable TCP/IP protocol:")
        print("   - Open SQL Server Configuration Manager")
        print("   - SQL Server Network Configuration → Protocols for SAURABH")
        print("   - Enable TCP/IP protocol")
        print("   - Restart SQL Server service\n")
        
        print("=" * 60)
        return False
        
    except Exception as e:
        print(f"✗ UNEXPECTED ERROR: {str(e)}\n")
        print("=" * 60)
        return False

if __name__ == "__main__":
    success = test_connection()
    sys.exit(0 if success else 1)

In [ ]:
from fastmcp import FastMCP
import os
import pyodbc

# Simple Windows Authentication - No password needed!
SERVER = 'DESKTOP-6IHV0U9\\SAURABH'
DATABASE = 'ExpenseTrackerDB'

CONNECTION_STRING = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={SERVER};DATABASE={DATABASE};Trusted_Connection=yes;'

mcp = FastMCP("ExpenseTracker")

def get_connection():
    """Create and return a database connection."""
    return pyodbc.connect(CONNECTION_STRING)

def init_db():
    """Initialize the database and create tables if they don't exist."""
    with get_connection() as conn:
        cursor = conn.cursor()
        cursor.execute("""
            IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='expenses' AND xtype='U')
            CREATE TABLE expenses(
                id INT PRIMARY KEY IDENTITY(1,1),
                date DATE NOT NULL,
                amount DECIMAL(10,2) NOT NULL,
                category NVARCHAR(100) NOT NULL,
                subcategory NVARCHAR(100) DEFAULT '',
                note NVARCHAR(500) DEFAULT ''
            )
        """)
        conn.commit()

init_db()

@mcp.tool()
def add_expense(date, amount, category, subcategory="", note=""):
    '''Add a new expense entry to the database.'''
    with get_connection() as conn:
        cursor = conn.cursor()
        cursor.execute(
            """INSERT INTO expenses(date, amount, category, subcategory, note) 
               VALUES (?, ?, ?, ?, ?)""",
            (date, amount, category, subcategory, note)
        )
        conn.commit()
        
        cursor.execute("SELECT @@IDENTITY AS id")
        row = cursor.fetchone()
        new_id = int(row[0]) if row else None
        
        return {"status": "ok", "id": new_id}
    


@mcp.tool()
def list_expenses(start_date, end_date):
    '''List all expenses between two dates (inclusive). Dates should be in YYYY-MM-DD format.'''
    with get_connection() as conn:
        cursor = conn.cursor()
        cursor.execute(
            """SELECT id, date, amount, category, subcategory, note 
               FROM expenses 
               WHERE date BETWEEN ? AND ? 
               ORDER BY date""",
            (start_date, end_date)
        )
        
        expenses = []
        for row in cursor.fetchall():
            expenses.append({
                "id": row.id,
                "date": str(row.date),
                "amount": float(row.amount),
                "category": row.category,
                "subcategory": row.subcategory,
                "note": row.note
            })
        
        return {"status": "ok", "count": len(expenses), "expenses": expenses}